In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Создадим таблицу с произвольными числами

При этом столбцы таблицы с прописными буквами A, B, C, D соответствуют ученикам, а строчные буквы a и b - количеству яблок и груш соответственно.

In [95]:
data = pd.DataFrame(np.random.randint(1,100, size=(10000,8)), columns=('Aa', 'Ab', 'Ba', 'Bb', 'Ca', 'Cb', 'Da', 'Db'))

In [96]:
data.head(3)

,Aa,Ab,Ba,Bb,Ca,Cb,Da,Db
0,11,3,53,79,58,42,23,63
1,91,53,5,66,68,54,42,27
2,16,64,78,44,87,7,94,95


Создадим промежуточную таблицу, где объединим количество яблок и груш в пары (для того, чтобы потом проще было обработать массив)

In [97]:
def joinColumns(x, label):
    return x[label+'a'], x[label+'b']

In [98]:
d = pd.DataFrame()
d['A'] = data.apply(lambda x: joinColumns(x, 'A'), axis=1)
d['B'] = data.apply(lambda x: joinColumns(x, 'B'), axis=1)
d['C'] = data.apply(lambda x: joinColumns(x, 'C'), axis=1)
d['D'] = data.apply(lambda x: joinColumns(x, 'D'), axis=1)

In [99]:
d.head(3)

,A,B,C,D
0,"(11, 3)","(53, 79)","(58, 42)","(23, 63)"
1,"(91, 53)","(5, 66)","(68, 54)","(42, 27)"
2,"(16, 64)","(78, 44)","(87, 7)","(94, 95)"


Функция определения счастливого ученика (если сортировка по первому элементу совпадает с обратной сортировкой по второму.

In [100]:
def isHappy(x):
    lst = list(x.to_numpy())
    a = sorted(lst, key=lambda x: x[0])
    b = sorted(lst, key=lambda x: x[1], reverse=True)
    if a==b:
        return 1;
    return 0;

Применяем функцию к преобразованному массиву и получаем столбец ответов: будет ли данная конфигурация учеников полностью счастливой

In [102]:
data1 = pd.DataFrame()
data1['y'] = d.apply(isHappy, axis=1)

In [121]:
data1.head(3)

,y
0,0
1,0
2,0


### Этап анализа

Разбиваем созданную выборку на обучающую и тестовую

In [104]:
X_train, X_test, y_train, y_test = train_test_split(data, data1['y'], test_size=0.3, random_state=42)

In [106]:
X_train.head(3)

,Aa,Ab,Ba,Bb,Ca,Cb,Da,Db
9069,5,14,52,35,6,66,98,54
2603,1,54,21,79,61,40,37,2
7738,11,40,4,4,79,45,78,67


Возьмем классификатор "случайный лес".
(Вообще говоря, это возможно не самый лучший вариант анализа и обучения, но то, что под руку попалось)

In [107]:
clf = RandomForestClassifier(random_state=0)

Обучение

In [108]:
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

Получаем вектор предсказания

In [109]:
pred=clf.predict(X_test)

In [110]:
pred

array([0, 0, 0, ..., 0, 0, 0])

Получаем точность предсказанного результата

In [111]:
accuracy_score(pred, y_test)

0.9503333333333334

Много выше, чем 0.5, но при этом и нулей в исходной последовательности больше.

А теперь попробуем наш единственный результат - ответ на задачу

In [116]:
test_row = {'Aa': [1], 'Ab': [4], 'Ba': [2], 'Bb': [3], 'Ca': [3], 'Cb': [2], 'Da': [4], 'Db':[1]}

In [117]:
test_df = pd.DataFrame(test_row)

In [118]:
test_df

,Aa,Ab,Ba,Bb,Ca,Cb,Da,Db
0,1,4,2,3,3,2,4,1


Предсказываем

In [120]:
clf.predict(test_df)

array([0])

Это фиаско!